In [ ]:
{-# LANGUAGE AllowAmbiguousTypes #-}
{-# LANGUAGE MultiParamTypeClasses #-}
{-# LANGUAGE KindSignatures #-}
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE FlexibleInstances #-}
{-# LANGUAGE TypeFamilies #-}

In [ ]:
import Data.String

In [ ]:
newtype Sky e = Sky e deriving (Show, Eq)
newtype Cosmo e = Cosmo e deriving (Show, Eq)
newtype Ground e = Ground e deriving (Show, Eq)
newtype Sea e = Sea e deriving (Show, Eq)

In [ ]:
data Air = Air deriving (Show, Eq)
data Water = Water deriving (Show, Eq)

In [ ]:
data JoinedForward a b = JoinedForward a b deriving (Show, Eq)
data JoinedBackward a b = JoinedBackward a b deriving (Show, Eq)

In [ ]:
a ~> b = JoinedForward a b
infixr ~>
a <~ b = JoinedBackward a b
infixl <~

In [ ]:
class ForwardLayer a e where
  type Backward a :: *
  forward :: Num e => a -> e -> (Backward a, e)

class BackwardLayer a e where
  type Forward a :: *
  backward :: Num e => a -> e -> (Forward a, e)

In [ ]:
instance ForwardLayer (Sky e) e where
  type Backward (Sky e) = Ground e
  forward (Sky e) e' = (Ground e, e + e')

In [ ]:
instance ForwardLayer (Cosmo e) e where
  type Backward (Cosmo e) = Sea e
  forward (Cosmo e) e' = (Sea e, e + e')

In [ ]:
instance ForwardLayer Air e where
  type Backward Air = Water
  forward Air e = (Water, e + 1)

In [ ]:
instance (ForwardLayer a e, ForwardLayer b e) => ForwardLayer (JoinedForward a b) e where
  type Backward (JoinedForward a b) = JoinedBackward (Backward a) (Backward b)
  forward (JoinedForward a b) e0 = (a' <~ b', e2)
      where
          (a', e1) = forward a e0
          (b', e2) = forward b e1

In [ ]:
instance BackwardLayer (Ground e) e where
  type Forward (Ground e) = Sky e
  backward (Ground e) e' = (Sky e, e - e')

In [ ]:
instance BackwardLayer (Sea e) e where
  type Forward (Sea e) = Cosmo e
  backward (Sea e) e' = (Cosmo e, e - e')

In [ ]:
instance BackwardLayer Water e where
  type Forward Water = Air
  backward Water e = (Air, e - 1)

In [ ]:
instance (BackwardLayer a e, BackwardLayer b e) => BackwardLayer (JoinedBackward a b) e where
  type Forward (JoinedBackward a b) = JoinedForward (Forward a) (Forward b)
  backward (JoinedBackward a b) e0 = (a' ~> b', e2)
    where
      (b', e1) = backward b e0
      (a', e2) = backward a e1

In [ ]:
forward (Sky 1) 2

In [ ]:
backward (Ground 2) 1

In [ ]:
uncurry backward $ forward (Sky 2) 1

In [ ]:
forward Air 3

In [ ]:
backward Water 4

In [ ]:
joined = Air ~> Sky 1 ~> Cosmo 2

In [ ]:
forward joined 4

In [ ]:
uncurry backward $ forward joined 4

In [ ]:
start :: (Num e, ForwardLayer a e, BackwardLayer (Backward a) e) => a -> (Backward a, e)
start a = forward a 0

In [ ]:
start (Sky 3)

In [ ]:
reverse :: (Num e, ForwardLayer a e, BackwardLayer (Backward a) e) => a -> e -> (Forward (Backward a), e)
reverse a e = uncurry backward $ forward a e

In [ ]:
reverse Air 0

In [ ]:
recursive a (e:es) = foldr f (a, [e]) es
  where
    f e (a, x:r) = let (n, v) = reverse a x in (n, v:x:r)

In [ ]:
recursive (Sky 2) [2, 3, 4]

In [ ]:
reverse (Sky 2) 2

In [ ]:
join :: (j) a -> [b] -> c
join y (x:xs) = join (x ~> Air ~> y) xs
join y [] = Air ~> y